<a href="https://colab.research.google.com/github/catmalark/Tensorflow-Study-Notes/blob/main/Tensorflow_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf


In [ ]:
x = tf.constant(4, shape = (1,1)) # 1x1 matrix
print(x)

tf.Tensor([[4]], shape=(1, 1), dtype=int32)


In [ ]:
x = tf.constant([[1, 2, 3], [4, 5, 6]])
print(x)

x = tf.ones((3, 3))
print(x)

x = tf.zeros((2, 4))
print(x)

x = tf.eye(3)    #identity matrix
print(x)

x = tf.random.normal((3,3), mean=0, stddev=1)  #normal distrubition
print(x)

x = tf.random.uniform((3,3), minval=0, maxval=1)
print(x)

In [ ]:
# Mathematical Operations
x = tf.constant([1,2,3])
y = tf.constant([9,8,7])
print(x + y)
print(x - y)
print(x / y)
print(x * y)

z = tf.tensordot(x, y, axes=1) #Element wise multipicaiton and summation of them
print(z)

z = x**5
print(z)

x = tf.random.normal((2, 3))
y = tf.random.normal((3, 4))
z = tf.matmul(x, y)
print(z)
z = x @ y
print(z)

In [ ]:
# Indexing
x = tf.constant([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
print(x[:])
print(x[1:]) #from 1.th element to end
print(x[1:3])
print(x[::-1])

indices = tf.constant([0, 3])
x_ind = tf.gather(x, indices) #0 and 3rd element
print(x_ind)

x = tf.constant([[1, 2],
                 [3, 4],
                 [5, 6]])
print(x[1, :]) # 1th indexed row, for us this is 2th row but computer its 1th
print(x[0:2, :])

In [ ]:
# Basic Reshaping
x = tf.range(9)
print(x)

x = tf.reshape(x, (3,3))
print(x)

x = tf.transpose(x)
print(x)

------------------------------------------------------------------------------

**Now Tutorial 3:**


In [ ]:
# Training Neural Network

import tensorflow as tf
from tensorflow import keras # offical API 
from keras import layers
from keras.datasets import mnist

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape) # this is currently 28x28 so we need to flatten them
print(y_train.shape)

x_train = x_train.reshape(-1, 784) / 255.0 # -1 means keep the first value in our case 60000 so it will be 60000x784
print(x_train.shape)
x_test = x_test.reshape(-1, 784) / 255.0 # Normalizing to make them in range 0-1

11501568/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
(60000, 784)


In [ ]:
#Sequential API (Very convient, not flexible)

model = keras.Sequential(
    [
     keras.Input(shape=(28*28)),   # For be able to write model summary 
     layers.Dense(512, activation="relu"),
     layers.Dense(216, activation="relu"),        # 2 hidden layers
     layers.Dense(10),
    ]
)

print(model.summary())     # to get a more info about model, helpfull for debugging 

model.compile(        # Tell keras how to configure the training part of our network
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True), #send to softmax
    optimizer = keras.optimizers.Adam(lr=0.001),    #lr is learning rate
    metrics = ["accuracy"],
)

model.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
model.evaluate(x_test, y_test, batch_size=32, verbose=2)



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 512)               401920    
                                                                 
 dense_4 (Dense)             (None, 216)               110808    
                                                                 
 dense_5 (Dense)             (None, 10)                2170      
                                                                 
Total params: 514,898
Trainable params: 514,898
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/5
1875/1875 - 10s - loss: 0.1860 - accuracy: 0.9436 - 10s/epoch - 6ms/step
Epoch 2/5
1875/1875 - 11s - loss: 0.0789 - accuracy: 0.9756 - 11s/epoch - 6ms/step
Epoch 3/5
1875/1875 - 13s - loss: 0.0550 - accuracy: 0.9825 - 13s/epoch - 7ms/step
Epoch 4/5
1875/1875 - 14s - loss: 0.0417 - accuracy: 0.9865 - 14s/epoch - 7ms/step
Epoch 5/5
1875/1875 - 14s - loss: 0.0322 - accuracy: 0.9897 - 14s/epoch - 8ms/step
313/313 - 1s - loss: 0.0842 - accuracy: 0.9793 - 1s/epoch - 4ms/step


[0.08422854542732239, 0.9793000221252441]

In [ ]:
#Functional API (More flexible, yu can put multiple inputs and outputs, whereas Sequentil API is 1 input to 1 output)

inputs = keras.Input(shape=(784))
x = layers.Dense(512, activation="relu")(inputs) #previus one we difined, sending the previous layer
x = layers.Dense(256, activation="relu")(x)       # you can also name layers with name=""
outputs = layers.Dense(10, activation="softmax")(x)  
model = keras.Model(inputs=inputs, outputs=outputs) # going to take inputs and outputs we defined above


model.compile(        
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False), #just beacuse we used softmax on outputs, from_logits will be false
    optimizer = keras.optimizers.Adam(learning_rate=0.001),    # try different optimizers than Adam, for exp: Adagrad, Gradient Descent with Momentum, RMSprop
    metrics = ["accuracy"],
)

model.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

Epoch 1/5
1875/1875 - 13s - loss: 0.1860 - accuracy: 0.9433 - 13s/epoch - 7ms/step
Epoch 2/5
1875/1875 - 11s - loss: 0.0779 - accuracy: 0.9754 - 11s/epoch - 6ms/step
Epoch 3/5
1875/1875 - 11s - loss: 0.0546 - accuracy: 0.9830 - 11s/epoch - 6ms/step
Epoch 4/5
1875/1875 - 10s - loss: 0.0398 - accuracy: 0.9874 - 10s/epoch - 6ms/step
Epoch 5/5
1875/1875 - 10s - loss: 0.0336 - accuracy: 0.9892 - 10s/epoch - 6ms/step
313/313 - 1s - loss: 0.0775 - accuracy: 0.9782 - 846ms/epoch - 3ms/step


[0.07748877257108688, 0.9782000184059143]

# Video 4:

In [14]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.datasets import cifar10    #50000 training images, 10000 test images, 32x32 pixels RGB

In [25]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [27]:
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [28]:
#Sequential API
from keras.layers.pooling import MaxPooling2D
model = keras.Sequential(
    [
     keras.Input(shape=(32, 32, 3)),   #We are not going to flat it in the begining 
     layers.Conv2D(32, 3, padding="valid", activation="relu"),    #How many channels this convolotional channel to output, 3 is (3,3)
     layers.MaxPooling2D(pool_size=(2,2)),
     layers.Conv2D(64, 3, activation="relu"),
     layers.MaxPooling2D(),
     layers.Conv2D(128, 3, activation="relu"),
     layers.Flatten(),
     layers.Dense(64, activation="relu"),
     layers.Dense(10),
    ]
)

print(model.summary()) 

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Nadam(learning_rate=3e-4),
    metrics=["accuracy"],
)

model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 15, 15, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_29 (Conv2D)          (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 6, 6, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_30 (Conv2D)          (None, 4, 4, 128)         73856     
                                                                 
 flatten_8 (Flatten)         (None, 2048)             

[622.1103515625, 0.15880000591278076]

In [17]:
#Now lets make a Functional API

def my_model():
  inputs = keras.Input(shape=(32,32,3))
  x = layers.Conv2D(32, 3)(inputs)
  x = layers.BatchNormalization()(x)
  x = keras.activations.relu(x)
  x = layers.MaxPooling2D()(x)
  x = layers.Conv2D(64, 5, padding="same")(x)
  x = layers.BatchNormalization()(x)
  x = keras.activations.relu(x)
  x = layers.Conv2D(128, 3)(x)
  x = layers.BatchNormalization()(x)
  x = keras.activations.relu(x)
  x = layers.Flatten()(x)
  x = layers.Dense(64, activation="relu")(x)
  outputs = layers.Dense(10)(x)
  model = keras.Model(inputs=inputs, outputs=outputs)
  return model

In [ ]:
model = my_model()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Nadam(learning_rate=3e-4),
    metrics=["accuracy"],
)

model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)